<a href="https://colab.research.google.com/github/nurshatfateh/WebScraping-NaturalLanguageProcessing/blob/main/trial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas
import spacy
from spacy.matcher import Matcher

In [ ]:
!pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import newspaper
from newspaper import Article

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def extract_article_from_url(url):
  a = Article(url)
  a.download()
  a.parse()
  return a.text, a.publish_date

In [ ]:
def sentence_giver(doc, start, end):
  s = start
  while str(doc[s]) != '.':
    s = s - 1
    if s < 0:
      break
  e = end
  while str(doc[e]) != '.':
    e = e + 1
    if e > len(doc) - 1:
      break
  return str(doc[s+1:e])

In [ ]:
def location_finder(doc):
  locations = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"ENT_TYPE": "GPE"}]
  matcher.add("Location", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = l_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      locations.append(doc[start:end])
  return locations, sentences

In [ ]:
def l_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LOWER": "accident"}]
  matcher.add("L_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) == 0:
    return 0
  else:
    return 1

In [ ]:
def number_finder(doc):
  numbers = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"LIKE_NUM": True}]
  matcher.add("Number", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = n_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      numbers.append(str(doc[start:end]))
  return numbers, sentences

In [ ]:
def n_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LOWER": "killed"}]
  matcher.add("N_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) == 0:
    return 0
  else:
    return 1

In [ ]:
link = 'https://www.ndtv.com/india-news/9-killed-in-keralas-palakkad-after-bus-carrying-school-students-crashes-into-state-transport-bus-3406641'
content, date = extract_article_from_url(link)
doc = nlp(content)

In [ ]:
l_places, l_s = location_finder(doc)
print(l_places[0])
print(l_s)

Kerala
['Kerala bus accident: Around 40 people were also injured in the crash, police said', '\n\nHe said the KSRTC bus was going from Kottarakkara in Kerala to Coimbatore in Tamil Nadu and had 81 passengers, of whom three died in the accident', '\n\nHe said the KSRTC bus was going from Kottarakkara in Kerala to Coimbatore in Tamil Nadu and had 81 passengers, of whom three died in the accident']


In [ ]:
n_people, n_s = number_finder(doc)
print(n_people[0])
print(n_s)

Nine
["\n\nNine people, including five students, were killed after a private tourist bus hit a state-run KSRTC bus from behind at Vadakkenchery in Kerala's Palakkad, Kerala Road Transport Minister Antony Raju said on Thursday", "\n\nNine people, including five students, were killed after a private tourist bus hit a state-run KSRTC bus from behind at Vadakkenchery in Kerala's Palakkad, Kerala Road Transport Minister Antony Raju said on Thursday", 'Nine persons, including five students and a teacher, were killed in the accident," he told the media', 'Nine persons, including five students and a teacher, were killed in the accident," he told the media']
